# Logging

**Sources:**

- [Fast api logging essentials](https://marketsplash.com/fastapi-logging/);

## Set up

In [1]:
from requests import get

The following cell runs a Docker container that utilizes a specified file to build a FastAPI application.

In [4]:
!docker run --rm -itd\
    --name test_container\
    -v ./logging_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app

ae3bdbf9389c684b2de43c453fdf5a583fc0078b508864064e465d0059c604ca


Don't forget to stop the container.

In [13]:
!docker stop test_container

test_container


## View logs

Let's see what the default logs of the fast api look like. We will make an application to write something to the console and then extract it from the docker container.

In the following cell we've just defined a simple fastapi program that throws a `HTTPException` if called with a `False` argument.

In [10]:
%%writefile logging_files/app.py
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.get("/{is_ok}")
def handle(is_ok : bool):
    if is_ok:
        return "im fine"
    else:
        raise HTTPException(status_code=404, detail="Item not found")

Overwriting logging_files/app.py


Now let's send some requests to the application and view it's log last lines:

In [6]:
get("http://localhost:8000/True")
get("http://localhost:8000/False")
!docker logs --tail 2 test_container

INFO:     172.17.0.1:34110 - "GET /True HTTP/1.1" 200 OK
INFO:     172.17.0.1:34114 - "GET /False HTTP/1.1" 404 Not Found


## Loggers

The following fastapi program returns all available loggers in the fast api.

In [11]:
%%writefile logging_files/app.py
import logging
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.get("/")
def handle():
    return [
        name for name in 
        logging.root.manager.loggerDict.keys()
    ]

Overwriting logging_files/app.py


So let's check what does we have.

In [12]:
response = get("http://localhost:8000")
response.content

b'["concurrent.futures","concurrent","asyncio","uvicorn.error","uvicorn","uvicorn.access","fastapi"]'

In my opinion, the most interesting logs for us here are `fastapi`, `unicorn`, `unicorn.error`, `unicorn.acess` and `fastapi`.